In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scip_workflows.common import *


In [ ]:
import fcsparser
import pyarrow


In [ ]:
try:
    ideas = snakemake.input.ideas_fcs
    output = snakemake.output[0]
except NameError:
    data_dir = Path("/home/maximl/scratch/data/vsc/datasets/wbc/ideas")
    ideas = data_dir / "fcs"
    output = data_dir / "WBC_ideas_features.parquet"


In [ ]:
# ideas features
def proc_cols(cols):
    out = []
    for col in cols:
        col = col.lower()
        for c in [" ", "-", "_", "."]:
            col = col.replace(c, "")
        out.append(col)
    return out


tmp = []
cols = None
for p in ideas.glob("*.fcs"):
    meta, data = fcsparser.parse(p)
    groups = re.search(
        "wbc(?P<group>[0-9])_(?P<part>[0-9])_?(?P<fix>(early|late))?.fcs", p.name
    ).groupdict()
    data["fix"] = groups["fix"]
    data["group"] = int(groups["group"])
    data["part"] = int(groups["part"])
    data.columns = proc_cols(data.columns)
    if cols is None:
        cols = data.columns
    tmp.append(data[cols])
df_ideas = pandas.concat(tmp, axis=0)

ideas_meta_cols = ["objectnumber", "fix", "group", "part"]
df_ideas = df_ideas.rename(
    columns=lambda col: f"meta_{col.lower()}"
    if col in ideas_meta_cols
    else f"feat_{col.lower()}"
)

df_ideas["meta_object_number"] = df_ideas["meta_objectnumber"].astype(int)
df_ideas = df_ideas.drop(columns="meta_objectnumber")

df_ideas = df_ideas.set_index(
    ["meta_group", "meta_part", "meta_fix", "meta_object_number"]
)


In [ ]:
r = r"(?i).*(uncompensated|bkgd|saturation|\().*"
todrop = list(filter(lambda x: re.match(r, x), df_ideas.columns))
df_ideas = df_ideas.drop(columns=todrop)
df_ideas.shape


In [ ]:
df_ideas.index = df_ideas.index.set_levels([0, 1, 2], level="meta_group")


In [ ]:
pq.write_table(pyarrow.Table.from_pandas(df_ideas), output)
